In [119]:
import pandas as pd
import os
import chardet
import re
import csv
import seaborn as sns
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

Data loaded from http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

#### USERS cleaning:
- estimate encoding here since its smaller database that still contains text
- remove outliers from age

In [2]:
# get encoding
data_path = os.path.join("data_dump", "BX-Users.csv")
rawdata = open(data_path, 'rb').read()
result = chardet.detect(rawdata)
result['encoding']

'ISO-8859-1'

In [120]:
data_path = os.path.join("data_dump", "BX-Users.csv")
users = pd.read_csv(data_path, encoding="ISO-8859-1", sep=";")
users.rename(columns={"User-ID": "user_ID"}, inplace=True)
users

,user_ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [93]:
print(users.dtypes)
print("\n")
print(users.isna().sum())

user_ID       int64
Location     object
Age         float64
dtype: object


user_ID          0
Location         0
Age         110762
dtype: int64


In [121]:
# remove outliers
users = users.loc[(users["Age"] > 7) & (users["Age"] < 90), :]

#### RATINGS cleaning:
- keep only users with more than 3 ratings (otherwise could be difficult to estimate pattern in preference) and keep only users with less than 1000 ratings (there is an user with 12000 ratings so some users_IDs might be rating aggregators or sth, 1000 seems like a plausible number, like in last ten years read 2 books per week) but both numbers are arbitrary and should be part of pipeline (like try a few options and evaluate on a sample)
- 40% of ratings are explicit, 60% implicit so we can’t discard  either group therefore for now implicit are converted to explicit with median value (if they bought the book, this is the most probable value of their reaction to it), but might change all to implicit in later analysis, this choice all implicit vs. all explicit should be also part of the pipeline

In [122]:
data_path = os.path.join("data_dump", "BX-Book-Ratings.csv")
ratings = pd.read_csv(data_path, encoding="ISO-8859-1", sep=";")
ratings.rename(columns = {'User-ID': "user_ID", "Book-Rating": "rating"}, inplace=True)
ratings

,user_ID,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [123]:
print(ratings.dtypes)
print("\n")
print(ratings.isna().sum())

user_ID     int64
ISBN       object
rating      int64
dtype: object


user_ID    0
ISBN       0
rating     0
dtype: int64


In [124]:
# remove users with not enough or too much ratings
rating_counts = ratings[["user_ID", "rating"]].groupby("user_ID").count().reset_index().sort_values("rating", ascending=False)
ratings = ratings.loc[ratings["user_ID"].isin(rating_counts.loc[(rating_counts["rating"] > 3) & (rating_counts["rating"] < 1000), "user_ID"]), :]

In [125]:
print(ratings["rating"].value_counts())

0     502187
8      76466
7      58165
10     57732
9      49350
5      37867
6      28361
4       6754
3       4508
2       2053
1       1192
Name: rating, dtype: int64


In [126]:
# how many explicit ratings?
print(ratings.loc[ratings["rating"]!=0,"rating"].count()/ratings.shape[0])

0.3910190569160901


In [127]:
ratings.loc[ratings["rating"]!=0,"rating"].median()

8.0

In [128]:
# impute implicit with median explicit
ratings.loc[ratings["rating"]==0,"rating"] = 8

C:\Users\Elena\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
ratings["rating"].value_counts()

8     578653
7      58165
10     57732
9      49350
5      37867
6      28361
4       6754
3       4508
2       2053
1       1192
Name: rating, dtype: int64

#### BOOKS cleaning
- about 20 books couldn't be parsed, problem in title so I made simple custom parser 
- csv reader has problem only when number of columns is higher than expected, so in 3 cases the number was lower (empty last column), these were duplicates so they were dropped
- still some missing values but they are not a problem
- some titles have multiple authors (up to 27) so title is not unique identifier 
- some popular titles have multiple ISBN (probably different editions and reprints) so ISBN can't be used for recommender

In [129]:
data_path = os.path.join("data_dump", "BX-Books.csv")
books = pd.read_csv(data_path, encoding="ISO-8859-1", sep=";", on_bad_lines="warn")
books.rename(columns = {'Book-Title': 'title', 
              'Book-Author': "author", 
              'Year-Of-Publication': "year", 
              'Publisher': "publisher", 
              "Image-URL-S":"url1", 
              "Image-URL-M": "url2", 
              "Image-URL-L": "url3"}, inplace=True)
books

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\Elena\anaconda3\lib\site-packages\IPython\core\interactiveshell.p

,ISBN,title,author,year,publisher,url1,url2,url3
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [130]:
# just to get problematic rows numbers:
missing_rows = []
with open(data_path) as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    for e, row in enumerate(reader):
        if (len(row) != 8):
            missing_rows.append(e)

# load rows in easier format:
f=open(data_path)
lines=f.readlines()

# parse these problematic rows:
missing_rows_content = []
for row_number in missing_rows:
    row_content = lines[row_number].split(";")
    row_dict = {}
    try:
        row_dict["year"] = eval(row_content[-5])
    except SyntaxError:
        print(row_number)
        continue     
    row_dict["ISBN"] = eval(row_content[0])
    row_dict["title"] = re.sub('[^A-Za-z0-9]+', ' ', row_content[1])
    row_dict["url3"] = eval(row_content[-1])
    row_dict["url2"] = eval(row_content[-2])
    row_dict["url1"] = eval(row_content[-3])
    row_dict["publisher"] = eval(row_content[-4])
    row_dict["author"] = eval(row_content[-6])
    missing_rows_content.append(row_dict)

missing_rows_df = pd.DataFrame(missing_rows_content)
books = pd.concat([missing_rows_df.reset_index(drop=True), books.reset_index(drop=True)], axis=0)

150788


In [69]:
# last column missings = some rows might not have parse ok
print(books.dtypes)
print("\n")
print(books.isna().sum())

year         object
ISBN         object
title        object
url3         object
url2         object
url1         object
publisher    object
author       object
dtype: object


year         0
ISBN         0
title        0
url3         3
url2         0
url1         0
publisher    2
author       1
dtype: int64


In [131]:
# rows that have parsed wrong
books.loc[books["url3"].isna(), :]
# books.loc[books["ISBN"]=="0789466953", :]
# books.loc[books["ISBN"]=="078946697X", :]
# books.loc[books["ISBN"]=="2070426769", :]

,year,ISBN,title,url3,url2,url1,publisher,author
209538,DK Publishing Inc,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",NaN,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,2000
220731,Gallimard,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",NaN,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,2003
221678,DK Publishing Inc,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",NaN,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,2000


In [132]:
# they are duplicates, so drop, did the rest parse ok?
books.drop(books.loc[books['url3'].isna()].index, inplace=True)
books.isna().sum()

year         0
ISBN         0
title        0
url3         0
url2         0
url1         0
publisher    2
author       1
dtype: int64

In [133]:
#some ISBN contain X, they all need to be in same format for merge
books["ISBN"] = books["ISBN"].astype(str) 

In [134]:
# drop irrelevant variables
books.drop(columns=["url1", "url2", "url3", "publisher", "year"], inplace=True)

In [135]:
# is ISBN unique?
books[["ISBN", "author"]].groupby("ISBN").nunique().sort_values("author", ascending=False).head(5)

,author
ISBN,
0000913154,1
0812515560,1
0812514440,1
0812514459,1
0812514475,1


In [75]:
# most old popular books = reprints
books[["title", "ISBN"]].groupby("title").nunique().sort_values("ISBN", ascending=False).head(10)

,ISBN
title,
Selected Poems,27
Little Women,24
Wuthering Heights,21
Adventures of Huckleberry Finn,20
Dracula,20
The Secret Garden,20
Jane Eyre,19
Pride and Prejudice,18
The Night Before Christmas,18


In [76]:
# is title unique?
books[["title", "author"]].groupby("title").nunique().sort_values("author", ascending=False).head(10)

,author
title,
Selected Poems,25
Dinosaurs,12
The Gift,12
The Secret,11
The Promise,11
Masquerade,11
Best Friends,11
Secrets,11
Journey,10


In [137]:
# just because it was book in the task but this needs to be done for other titles too
# bottlenect of the whole analysis!
# solution probably includes NLP like get all titles together, filter out common words across authors, ?
books.loc[books.title.str.lower().str.contains(pat='lord of the rings') & books.author.str.lower().str.replace('\W', '').str.contains(pat='jrrtolkien'), "title"] = "The Lord of the Rings"

<ipython-input-137-26f76e3b9c46>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  books.loc[books.title.str.lower().str.contains(pat='lord of the rings') & books.author.str.lower().str.replace('\W', '').str.contains(pat='jrrtolkien'), "title"] = "The Lord of the Rings"


In [139]:
# new unique (in the future) identifier
books["authortitle"] = books["author"].str.lower().str.replace('\W', '') + books["title"].str.lower().str.replace('\W', '')

<ipython-input-139-63e8d78fb435>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  books["authortitle"] = books["author"].str.lower().str.replace('\W', '') + books["title"].str.lower().str.replace('\W', '')


#### Connect the datasets:
- merge all on ISBN (inner) and user_ID (not inner)
- should age be included in making recommendations? spearman correlations between rating and age separately for every book that has at least 50 ratings, then median of absolute values => low value = no 
- ISBN can't be used for recommender so new identifier created as concatenation of author and title
- keep only max rating of user_ID for given authortitle identifier (users rated different editions)
- drop books with less than 5 ratings (arbitrary number, should be part of pipeline)
- make sparse pivot and save

In [140]:
# ISBN not matched: 0385258259, 15655122046, 0140260676 - search in books.csv didnt find them either
# reprints have different ISBN
# from this code df[["ISBN", "rating"]].groupby(["ISBN"]).count().sort_values("rating", ascending=False).reset_index().head(10).merge(books[["ISBN", "title"]], on="ISBN")
df = books.merge(ratings, on="ISBN", how="inner")
df = df.merge(users, on="user_ID", how="left")

In [141]:
# does age play role? No.
corr_df=df.loc[:,["rating", "Age", "authortitle"]].groupby("authortitle").corr("spearman").reset_index(drop=False)
corr_df = corr_df.loc[corr_df["level_1"]=="Age", :]

counts_df = df.loc[:,["rating", "Age", "authortitle"]].groupby("authortitle").count().reset_index(drop=False)
corr_df = corr_df.loc[corr_df["authortitle"].isin(counts_df.loc[counts_df["Age"]>50, "authortitle"]), "rating"]
corr_df.abs().median()

0.07752263788555638

In [142]:
# max in ISBN has no meaning so just choose one, choose max rating
df = df[["authortitle","ISBN", "rating", "user_ID"]].groupby(["authortitle", "user_ID"]).max().reset_index()

In [143]:
# drop books with less than 5 ratings, probably will drop more in future analysis
ratings_info = df[["rating", "authortitle"]].groupby("authortitle").count().reset_index(drop=False)
df = df.loc[df["authortitle"].isin(ratings_info.loc[ratings_info["rating"] >=5, "authortitle"]), :]

In [144]:
df_pivot=df.pivot(index="user_ID", columns="authortitle", values="rating")
df_pivot=df_pivot.fillna(0)
pivot_sparse = csr_matrix(df_pivot)

In [150]:
books = books[["authortitle", "author", "title"]].groupby("authortitle").min().reset_index()
books["is_analysed"] = 0
books.loc[books["authortitle"].isin(df_pivot.columns), "is_analysed"] = 1
books.to_csv("books_cleaned.csv")

In [146]:
with open(r"df_columns.pickle", "wb") as output_file:
    pickle.dump(df_pivot.columns, output_file)
with open(r"df_index.pickle", "wb") as output_file:
    pickle.dump(df_pivot.index, output_file)
with open(r"df_pivot_sparse.pickle", "wb") as output_file:
    pickle.dump(pivot_sparse, output_file)